In [102]:
import pandas as pd
import numpy as np

import math
import time

from tqdm import tqdm

def add_last_problem(df):
    new = []
    pre = df["testId"][0]
    for idx in df["testId"]:
        if pre != idx:
            new[-1] = -1
            pre = idx
        new.append(0)
    df["last_problem"] = new
    return df

dtype = {"userID": "int16", "answerCode": "int8", "KnowledgeTag": "int16"}

df = pd.read_csv('../../data/test_data.csv', dtype=dtype, parse_dates=["Timestamp"])
df = df.sort_values(by=["userID", "Timestamp"]).reset_index(drop=True)

df["problem_number"] = df["assessmentItemID"].apply(lambda x: int(x[-3:]))

correct_t = df.groupby(["testId"])["answerCode"].agg(["mean", "sum"])
correct_t.columns = ["test_mean", "test_sum"]
correct_k = df.groupby(["KnowledgeTag"])["answerCode"].agg(["mean", "sum"])
correct_k.columns = ["tag_mean", "tag_sum"]
correct_a = df.groupby(["assessmentItemID"])["answerCode"].agg(["mean", "sum"])
correct_a.columns = ["ass_mean", "ass_sum"]
correct_p = df.groupby(["problem_number"])["answerCode"].agg(["mean", "sum"])
correct_p.columns = ["prb_mean", "prb_sum"]

In [103]:
def feature_engineering(df):
    print("-" * 20, "Feature Engineering Start", "-" * 20)
    start_time = time.time()
    # 유저별 시퀀스를 고려하기 위해 아래와 같이 정렬
    df.sort_values(by=["userID", "Timestamp"], inplace=True)
    df = add_last_problem(df)
    # elo 추가
    # df = ELO_function(df)

    df["hour"] = df["Timestamp"].dt.hour
    df["dow"] = df["Timestamp"].dt.dayofweek

    # 푸는 시간
    diff = (
        df.loc[:, ["userID", "Timestamp"]]
        .groupby("userID")
        .diff()
        .fillna(pd.Timedelta(seconds=0))
    )
    diff = diff.fillna(pd.Timedelta(seconds=0))
    diff = diff["Timestamp"].apply(lambda x: x.total_seconds())
    df["elapsed"] = diff
    df["elapsed"] = df["elapsed"].apply(lambda x: x if x < 600 and x >= 0 else 0)
    def elap(x):
        if x == 0:
            return 0
        elif x < 9:
            return 1
        elif x < 101:
            return 2
        elif x < 201:
            return 3
        elif x < 301:
            return 4
        elif x < 401:
            return 5
        elif x < 501:
            return 6
        elif x < 601:
            return 7
        
    df.elapsed = df.elapsed.apply(lambda x : elap(x))

    df["grade"] = df["testId"].apply(lambda x: int(x[2]))
    df["mid"] = df["testId"].apply(lambda x: x[7:10])
    df["problem_number"] = df["assessmentItemID"].apply(lambda x: int(x[-3:]))
    df['assessmentItemID0'] = df['assessmentItemID'].str[2]
    df['assessmentItemID1'] = df['assessmentItemID'].str[4:7]
    
    correct_h = df.groupby(["hour"])["answerCode"].agg(["mean", "sum"])
    correct_h.columns = ["hour_mean", "hour_sum"]
    correct_d = df.groupby(["dow"])["answerCode"].agg(["mean", "sum"])
    correct_d.columns = ["dow_mean", "dow_sum"]

    df = pd.merge(df, correct_t, on=["testId"], how="left")
    df = pd.merge(df, correct_k, on=["KnowledgeTag"], how="left")
    df = pd.merge(df, correct_a, on=["assessmentItemID"], how="left")
    df = pd.merge(df, correct_p, on=["problem_number"], how="left")
    df = pd.merge(df, correct_h, on=["hour"], how="left")
    df = pd.merge(df, correct_d, on=["dow"], how="left")

    o_df = df[df["answerCode"] == 1]
    x_df = df[df["answerCode"] == 0]

    elp_k = df.groupby(["KnowledgeTag"])["elapsed"].agg("mean").reset_index()
    elp_k.columns = ["KnowledgeTag", "tag_elp"]
    elp_k_o = o_df.groupby(["KnowledgeTag"])["elapsed"].agg("mean").reset_index()
    elp_k_o.columns = ["KnowledgeTag", "tag_elp_o"]
    elp_k_x = x_df.groupby(["KnowledgeTag"])["elapsed"].agg("mean").reset_index()
    elp_k_x.columns = ["KnowledgeTag", "tag_elp_x"]

    df = pd.merge(df, elp_k, on=["KnowledgeTag"], how="left")
    df = pd.merge(df, elp_k_o, on=["KnowledgeTag"], how="left")
    df = pd.merge(df, elp_k_x, on=["KnowledgeTag"], how="left")

    ass_k = df.groupby(["assessmentItemID0"])["elapsed"].agg("mean").reset_index()
    ass_k.columns = ["assessmentItemID0", "ass_elp"]
    ass_k_o = o_df.groupby(["assessmentItemID0"])["elapsed"].agg("mean").reset_index()
    ass_k_o.columns = ["assessmentItemID0", "ass_elp_o"]
    ass_k_x = x_df.groupby(["assessmentItemID0"])["elapsed"].agg("mean").reset_index()
    ass_k_x.columns = ["assessmentItemID0", "ass_elp_x"]

    df = pd.merge(df, ass_k, on=["assessmentItemID0"], how="left")
    df = pd.merge(df, ass_k_o, on=["assessmentItemID0"], how="left")
    df = pd.merge(df, ass_k_x, on=["assessmentItemID0"], how="left")

    prb_k = df.groupby(["problem_number"])["elapsed"].agg("mean").reset_index()
    prb_k.columns = ["problem_number", "prb_elp"]
    prb_k_o = o_df.groupby(["problem_number"])["elapsed"].agg("mean").reset_index()
    prb_k_o.columns = ["problem_number", "prb_elp_o"]
    prb_k_x = x_df.groupby(["problem_number"])["elapsed"].agg("mean").reset_index()
    prb_k_x.columns = ["problem_number", "prb_elp_x"]

    df = pd.merge(df, prb_k, on=["problem_number"], how="left")
    df = pd.merge(df, prb_k_o, on=["problem_number"], how="left")
    df = pd.merge(df, prb_k_x, on=["problem_number"], how="left")

    df["user_correct_answer"] = (
        df.groupby("userID")["answerCode"]
        .transform(lambda x: x.cumsum().shift(1))
        .fillna(0)
    )
    df["user_total_answer"] = df.groupby("userID")["answerCode"].cumcount()
    df["user_acc"] = (df["user_correct_answer"] / df["user_total_answer"]).fillna(0)
    df["Grade_o"] = (
        df.groupby(["userID", "grade"])["answerCode"]
        .transform(lambda x: x.cumsum().shift(1))
        .fillna(0)
    )
    df["GradeCount"] = df.groupby(["userID", "grade"]).cumcount()
    df["GradeAcc"] = (df["Grade_o"] / df["GradeCount"]).fillna(0)
    df["GradeElp"] = (
        df.groupby(["userID", "grade"])["elapsed"]
        .transform(lambda x: x.cumsum())
        .fillna(0)
    )
    df["GradeMElp"] = df["GradeElp"] / [
        v if v != 0 else 1 for v in df["GradeCount"].values
    ]

    f = lambda x: len(set(x))
    test = df.groupby(["testId"]).agg({"problem_number": "max", "KnowledgeTag": f})
    test.reset_index(inplace=True)

    test.columns = ["testId", "problem_count", "tag_count"]

    df = pd.merge(df, test, on="testId", how="left")

    gdf = df[["userID", "testId", "problem_number", "grade", "Timestamp"]].sort_values(
        by=["userID", "grade", "Timestamp"]
    )
    gdf["buserID"] = gdf["userID"] != gdf["userID"].shift(1)
    gdf["bgrade"] = gdf["grade"] != gdf["grade"].shift(1)
    gdf["first"] = gdf[["buserID", "bgrade"]].any(axis=1).apply(lambda x: 1 - int(x))
    gdf["RepeatedTime"] = gdf["Timestamp"].diff().fillna(pd.Timedelta(seconds=0))
    gdf["RepeatedTime"] = (
        gdf["RepeatedTime"].apply(lambda x: x.total_seconds()) * gdf["first"]
    )
    df["RepeatedTime"] = gdf["RepeatedTime"].apply(lambda x: math.log(x + 1))

    df["prior_KnowledgeTag_frequency"] = df.groupby(
        ["userID", "KnowledgeTag"]
    ).cumcount()

    df["problem_position"] = df["problem_number"] / df["problem_count"]
    df["solve_order"] = df.groupby(["userID", "testId"]).cumcount()
    df["solve_order"] = (
        df["solve_order"]
        - df["problem_count"] * (df["solve_order"] > df["problem_count"]).apply(int)
        + 1
    )
    df["retest"] = (df["solve_order"] > df["problem_count"]).apply(int)
    T = df["solve_order"] != df["problem_number"]
    TT = T.shift(1)
    TT[0] = False
    df["solved_disorder"] = (TT.apply(lambda x: not x) & T).apply(int)

    #df["testId"] = df["testId"].apply(lambda x: int(x[1:4] + x[-3]))

    print("-" * 20, "Feature Engineering End", "-" * 20)
    print(f"Feature Engineering에 걸린 시간 : {time.time() - start_time}s")
    return df

In [104]:
df = feature_engineering(df)
df

-------------------- Feature Engineering Start --------------------
-------------------- Feature Engineering End --------------------
Feature Engineering에 걸린 시간 : 6.185323476791382s


,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,problem_number,last_problem,hour,dow,...,GradeElp,GradeMElp,problem_count,tag_count,RepeatedTime,prior_KnowledgeTag_frequency,problem_position,solve_order,retest,solved_disorder
0,3,A050023001,A050000023,1,2020-01-09 10:56:31,2626,1,0,10,3,...,0,0.000000,7,4,0.000000,0,0.142857,1,0,0
1,3,A050023002,A050000023,1,2020-01-09 10:56:57,2626,2,0,10,3,...,2,2.000000,7,4,3.295837,1,0.285714,2,0,0
2,3,A050023003,A050000023,0,2020-01-09 10:58:31,2625,3,0,10,3,...,4,2.000000,7,4,4.553877,0,0.428571,3,0,0
3,3,A050023004,A050000023,0,2020-01-09 10:58:36,2625,4,0,10,3,...,5,1.666667,7,4,1.791759,1,0.571429,4,0,0
4,3,A050023006,A050000023,0,2020-01-09 10:58:43,2623,6,0,10,3,...,6,1.500000,7,4,2.079442,0,0.857143,5,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260109,7439,A040130001,A040000130,0,2020-10-14 23:07:23,8832,1,0,23,2,...,17,1.545455,5,2,15.367586,0,0.200000,1,0,0
260110,7439,A040130002,A040000130,1,2020-10-14 23:07:41,8832,2,0,23,2,...,19,1.583333,5,2,2.944439,1,0.400000,2,0,0
260111,7439,A040130003,A040000130,1,2020-10-14 23:08:02,8244,3,0,23,2,...,21,1.615385,5,2,3.091042,0,0.600000,3,0,0
260112,7439,A040130004,A040000130,1,2020-10-14 23:09:31,8244,4,0,23,2,...,23,1.642857,5,2,4.499810,1,0.800000,4,0,0


In [105]:
train = pd.read_csv('../../data/test_data.csv', dtype=dtype, parse_dates=["Timestamp"])
train = train.sort_values(by=["userID", "Timestamp"]).reset_index(drop=True)
df["problem_number"] = train["assessmentItemID"].apply(lambda x: x[-3:])

In [101]:
df.to_csv('../../data/4.0_for_label.csv', index=False)

In [106]:
import time
from datetime import datetime
df2 = pd.read_csv('../../data/test_data.csv')
df2 = df2.sort_values(by=["userID", "Timestamp"]).reset_index(drop=True)
df['Timestamp'] = df2['Timestamp']
def convert_time(s):
    timestamp = time.mktime(
        datetime.strptime(s, "%Y-%m-%d %H:%M:%S").timetuple()
    )
    return int(timestamp)

df['Timestamp'] = df.Timestamp.apply(convert_time)

In [107]:
df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,problem_number,last_problem,hour,dow,...,GradeElp,GradeMElp,problem_count,tag_count,RepeatedTime,prior_KnowledgeTag_frequency,problem_position,solve_order,retest,solved_disorder
0,3,A050023001,A050000023,1,1578567391,2626,001,0,10,3,...,0,0.000000,7,4,0.000000,0,0.142857,1,0,0
1,3,A050023002,A050000023,1,1578567417,2626,002,0,10,3,...,2,2.000000,7,4,3.295837,1,0.285714,2,0,0
2,3,A050023003,A050000023,0,1578567511,2625,003,0,10,3,...,4,2.000000,7,4,4.553877,0,0.428571,3,0,0
3,3,A050023004,A050000023,0,1578567516,2625,004,0,10,3,...,5,1.666667,7,4,1.791759,1,0.571429,4,0,0
4,3,A050023006,A050000023,0,1578567523,2623,006,0,10,3,...,6,1.500000,7,4,2.079442,0,0.857143,5,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260109,7439,A040130001,A040000130,0,1602716843,8832,001,0,23,2,...,17,1.545455,5,2,15.367586,0,0.200000,1,0,0
260110,7439,A040130002,A040000130,1,1602716861,8832,002,0,23,2,...,19,1.583333,5,2,2.944439,1,0.400000,2,0,0
260111,7439,A040130003,A040000130,1,1602716882,8244,003,0,23,2,...,21,1.615385,5,2,3.091042,0,0.600000,3,0,0
260112,7439,A040130004,A040000130,1,1602716971,8244,004,0,23,2,...,23,1.642857,5,2,4.499810,1,0.800000,4,0,0


In [113]:
cate = ['assessmentItemID', 'testId', 
       'KnowledgeTag', 'problem_number', 'last_problem', 'hour', 'dow',
       'elapsed', 'grade', 'mid', 'assessmentItemID0', 'assessmentItemID1',
        'solve_order', 'solved_disorder', 'userID2']
conti = ['test_mean', 'test_sum', 'tag_mean', 'tag_sum','ass_mean', 'ass_sum',
         'prb_mean', 'prb_sum', 'hour_mean', 'hour_sum', 'dow_mean', 'dow_sum',
        'tag_elp', 'tag_elp_o', 'tag_elp_x', 'ass_elp', 'ass_elp_o',
        'ass_elp_x', 'prb_elp', 'prb_elp_o', 'prb_elp_x','user_correct_answer',
        'user_total_answer', 'user_acc', 'Grade_o', 'GradeCount', 'GradeAcc',
        'GradeElp', 'GradeMElp','RepeatedTime', 'prior_KnowledgeTag_frequency',
        'problem_count', 'tag_count', 'prior_KnowledgeTag_frequency', 'problem_position'
       ]
#df2 = pd.read_csv('../../data/4.0_for_label.csv')
df['userID2'] = df['userID']
for i in cate:
    cate2label = {j:i for i,j in enumerate(df[i].unique())}
    df[i] = df[i].map(cate2label)

df.to_csv('../../data/user.csv')

In [3]:
import pandas as pd
df = pd.read_csv('../../data/user.csv')
df.columns

Index(['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
       'KnowledgeTag', 'problem_number', 'last_problem', 'hour', 'dow',
       'elapsed', 'grade', 'mid', 'assessmentItemID0', 'assessmentItemID1',
       'test_mean', 'test_sum', 'tag_mean', 'tag_sum', 'ass_mean', 'ass_sum',
       'prb_mean', 'prb_sum', 'hour_mean', 'hour_sum', 'dow_mean', 'dow_sum',
       'tag_elp', 'tag_elp_o', 'tag_elp_x', 'ass_elp', 'ass_elp_o',
       'ass_elp_x', 'prb_elp', 'prb_elp_o', 'prb_elp_x', 'user_correct_answer',
       'user_total_answer', 'user_acc', 'Grade_o', 'GradeCount', 'GradeAcc',
       'GradeElp', 'GradeMElp', 'problem_count', 'tag_count', 'RepeatedTime',
       'prior_KnowledgeTag_frequency', 'problem_position', 'solve_order',
       'retest', 'solved_disorder', 'userID2'],
      dtype='object')

In [5]:
df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,problem_number,last_problem,hour,dow,...,GradeMElp,problem_count,tag_count,RepeatedTime,prior_KnowledgeTag_frequency,problem_position,solve_order,retest,solved_disorder,userID2
0,0,0,0,1,2020-03-24 00:17:11,0,0,0,0,0,...,0.00,7,2,0.000000,0,0.142857,0,0,0,0
1,0,1,0,1,2020-03-24 00:17:14,1,1,0,0,0,...,1.00,7,2,1.386294,0,0.285714,1,0,0,0
2,0,2,0,1,2020-03-24 00:17:22,1,2,0,0,0,...,1.00,7,2,2.197225,1,0.428571,2,0,0,0
3,0,3,0,1,2020-03-24 00:17:29,1,3,0,0,0,...,1.00,7,2,2.079442,2,0.571429,3,0,0,0
4,0,4,0,1,2020-03-24 00:17:36,1,4,0,0,0,...,1.00,7,2,2.079442,3,0.714286,4,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2525951,7441,4499,713,0,2020-06-05 06:50:21,667,4,1,22,4,...,2.25,5,1,3.218876,4,1.000000,4,0,0,7441
2525952,7441,1286,199,1,2020-08-21 01:06:39,274,0,0,3,4,...,0.00,4,1,0.000000,0,0.250000,0,0,0,7441
2525953,7441,1287,199,1,2020-08-21 01:06:50,274,1,0,3,4,...,2.00,4,1,2.484907,1,0.500000,1,0,0,7441
2525954,7441,1288,199,1,2020-08-21 01:07:36,274,2,0,3,4,...,2.00,4,1,3.850148,2,0.750000,2,0,0,7441


In [72]:
def __augmentation(df):
    cnt = pd.DataFrame(df.groupby('userID')['answerCode'].count())
    cnt.answerCode = cnt.answerCode//100
    i_cnt = cnt[cnt['answerCode'] >= 1]

    n_index = -1
    for i,j in enumerate(i_cnt.index):
        c = i_cnt.iloc[i].answerCode
        index = df[df['userID'] == j].index
        for k in range(2, c+2):
            ii = index[-k*100:-100*(k-1)]
            if len(ii) > 20:
                df.loc[ii, ['userID']] = n_index
                n_index -= 1
    return df

df = __augmentation(df)
df.to_csv('../../data/train_test_data_4.0.csv', index=False)

In [109]:
df.to_csv('../../data/test_data_4.0.csv', index=False)

In [112]:
df = pd.read_csv('../../data/4.0_for_label.csv')
df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,problem_number,last_problem,hour,dow,...,GradeElp,GradeMElp,problem_count,tag_count,RepeatedTime,prior_KnowledgeTag_frequency,problem_position,solve_order,retest,solved_disorder
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1,0,0,1,...,0,0.00,7,2,0.000000,0,0.142857,1,0,0
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,2,0,0,1,...,1,1.00,7,2,1.386294,0,0.285714,2,0,0
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,3,0,0,1,...,2,1.00,7,2,2.197225,1,0.428571,3,0,0
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,4,0,0,1,...,3,1.00,7,2,2.079442,2,0.571429,4,0,0
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,5,0,0,1,...,4,1.00,7,2,2.079442,3,0.714286,5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2525951,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,5,-1,6,4,...,9,2.25,5,1,3.218876,4,1.000000,5,0,0
2525952,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,1,0,1,4,...,0,0.00,4,1,0.000000,0,0.250000,1,0,0
2525953,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,2,0,1,4,...,2,2.00,4,1,2.484907,1,0.500000,2,0,0
2525954,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,3,0,1,4,...,4,2.00,4,1,3.850148,2,0.750000,3,0,0
